# Create a runnable with the @chain decorator
### LangChain Expression Language (LCEL)

---

Alejandro Ricciardi (Omegapy)  
created date: 01/21/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/) 

<br>

---

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. 

**In this project:**  
I explore:
- @chain: 
The concept of decoration, ```@chain```, to arbitrary function.
- Chain Memory - Add message history:
The ```RunnableWithMessageHistory``` class implements message history to certain types of chains.





<p></p>
<b style="font-size:15;">
⚠️ This project requires an OpenAi key.
</b>


Project Map:
- [API Key](#api-key)
- [@Chain](#chain)
- [Chain Memory - Add message history](#chain-memory---add-message-history)

        
- []()
    

<br>

---


### API Key

In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")

[Project Map](#project-map)

---

---
## @Chain

Arbitrary function can be turn into a chain by adding a ```@chain``` decorator.  
This is functionally equivalent to wrapping in a [RunnableLambda - Run Custom Functions](https://python.langchain.com/docs/expression_language/how_to/functions).

This will have the benefit of improved observability by tracing your chain correctly.  
Any calls to runnables inside this function will be traced as nested children.
It will also allow you to use this as any other runnable, compose it in chain, etc.

<br>

---

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

In [3]:
prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

#### Arbitrary Function (Custom Chain)

In [4]:
@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

#### The ```custom_chain``` is now a runnable, meaning you will need to use ```invoke```

In [5]:
custom_chain.invoke("bears")

'The subject of this joke is bears.'

If you check out your LangSmith traces, you should see a ```custom_chain``` trace in there, with the calls to OpenAI nested underneath

[Project Map](#project-map)

---

---
## Chain Memory - Add message history

The ```RunnableWithMessageHistory``` let’s us add message history to certain types of chains.

Specifically, it can be used for any Runnable that takes as input one of
- a sequence of BaseMessage
- a dict with a key that takes a sequence of ```BaseMessage```
- a dict with a key that takes the latest message(s) as a string or sequence of ```BaseMessage```, and a separate key that takes historical messages
And returns as output one of
- a string that can be treated as the contents of an ```AIMessage```
- a sequence of ```BaseMessage```
- a dict with a key that contains a sequence of ```BaseMessage```

<br>

---

### Setup

I will use [Redis](https://redis.io/) to store the chat message histories and OpenAi models.  
Dependencies:

In [6]:
%pip install --upgrade --quiet  langchain redis openai

Note: you may need to restart the kernel to use updated packages.


In [9]:

import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

Start a local Redis Stack server if we don’t have an existing Redis deployment to connect to:

- You need to install [docker](https://docs.docker.com/get-started/overview/) in your desktop: [Install Docker Desktop on Windows](https://docs.docker.com/desktop/install/windows-install/)
Docker is an open platform for developing, shipping, and running applications. Docker enables you to separate your applications from your infrastructure so you can deliver software quickly. With Docker, you can manage your infrastructure in the same ways you manage your applications. By taking advantage of Docker's methodologies for shipping, testing, and deploying code, you can significantly reduce the delay between writing code and running it in production.

In [10]:
!docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest

docker: request returned Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2Fdocker_engine/v1.24/containers/create, check if the server supports the requested API version.
See 'docker run --help'.
